In [1]:
import os
import sys

import msprime

from dask.distributed import Client
from dask.distributed import LocalCluster


# import local phylokit modules
phylokit_path = os.path.abspath(os.path.join(os.pardir))
if phylokit_path not in sys.path:
    sys.path.append(phylokit_path)

import phylokit as pk

# Setup Client

In [2]:
localcluster = LocalCluster(n_workers=5, threads_per_worker=2)
client = Client(localcluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52836,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:53074,Total threads: 2
Dashboard: http://127.0.0.1:53078/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:52843,


In [3]:
ts = msprime.sim_ancestry(10000, sequence_length=100000, random_seed=1234)
mts = msprime.sim_mutations(ts, rate=0.01, random_seed=5678)
pk_mts = pk.from_tskit(mts.first())
pk_mts

<xarray.Dataset>
Dimensions:              (nodes: 40000, samples: 20000, traversal: 39999)
Dimensions without coordinates: nodes, samples, traversal
Data variables:
    node_parent          (nodes) int32 22682 24045 24897 26556 ... 39998 -1 -1
    node_left_child      (nodes) int32 -1 -1 -1 -1 ... 39992 39905 39996 39998
    node_right_sib       (nodes) int32 4726 3804 17477 26189 ... 39997 -1 -1 -1
    sample_node          (samples) int32 0 1 2 3 4 ... 19996 19997 19998 19999
    traversal_preorder   (traversal) int32 39998 39996 39992 ... 22425 7006 8530
    traversal_postorder  (traversal) int32 11392 18988 23411 ... 39997 39998
    node_time            (nodes) float64 0.0 0.0 0.0 0.0 ... 2.289 2.545 inf
    node_branch_length   (nodes) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 1 0 0 0

In [4]:
%time _ = pk.get_hartigan_parsimony_score(pk_mts, mts.genotype_matrix(), ["A", "C", "T", "G"], chunk_size=1200)
pk.get_hartigan_parsimony_score(pk_mts, mts.genotype_matrix(), ["A", "C", "T", "G"], chunk_size=1200)

CPU times: user 2.03 s, sys: 384 ms, total: 2.41 s
Wall time: 7.96 s


<xarray.DataArray (sites: 32558)>
array([1, 1, 1, ..., 1, 1, 2], dtype=int32)
Dimensions without coordinates: sites

In [6]:
%time _ = pk.numba_hartigan_parsimony_vectorised(pk_mts.node_left_child.data,pk_mts.node_right_sib.data,pk_mts.sample_node.data,mts.genotype_matrix(),["A", "C", "G", "T"],)
pk.numba_hartigan_parsimony_vectorised(pk_mts.node_left_child.data,pk_mts.node_right_sib.data,pk_mts.sample_node.data,mts.genotype_matrix(),["A", "C", "G", "T"])

CPU times: user 15.8 s, sys: 2.66 s, total: 18.5 s
Wall time: 18.6 s


array([1, 1, 1, ..., 1, 1, 2], dtype=int32)

In [7]:
%time _ = pk.append_parsimony_score(pk_mts, mts.genotype_matrix(), ["A", "C", "T", "G"], chunk_size=1200)
pk.append_parsimony_score(pk_mts, mts.genotype_matrix(), ["A", "C", "T", "G"], chunk_size=1200)

CPU times: user 2.08 s, sys: 371 ms, total: 2.45 s
Wall time: 6.97 s


<xarray.Dataset>
Dimensions:                (nodes: 40000, samples: 20000, traversal: 39999, sites: 32558)
Dimensions without coordinates: nodes, samples, traversal, sites
Data variables:
    node_parent            (nodes) int32 22682 24045 24897 26556 ... 39998 -1 -1
    node_left_child        (nodes) int32 -1 -1 -1 -1 ... 39992 39905 39996 39998
    node_right_sib         (nodes) int32 4726 3804 17477 26189 ... -1 -1 -1
    sample_node            (samples) int32 0 1 2 3 4 ... 19996 19997 19998 19999
    traversal_preorder     (traversal) int32 39998 39996 39992 ... 7006 8530
    traversal_postorder    (traversal) int32 11392 18988 23411 ... 39997 39998
    node_time              (nodes) float64 0.0 0.0 0.0 0.0 ... 2.289 2.545 inf
    node_branch_length     (nodes) int32 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 1 0 0 0
    sites_genotypes        (sites, samples) int32 0 0 0 0 0 1 0 ... 0 0 0 0 0 0
    sites_parsimony_score  (sites) int32 1 1 1 1 1 1 1 1 1 ... 2 1 1 2 1 1 1 1 2